In [14]:
# import comet_ml in the top of your file
# from comet_ml import Experiment
    
# # Add the following code anywhere in your machine learning file
# experiment = Experiment(api_key="8kbyc7YDajZyTL0alFDxawj8c",
#                         project_name="sonidos", workspace="hzlclock")
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, SimpleRNN, LSTM, GRU, Lambda, Permute
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.layers import Conv2D, MaxPooling2D,AveragePooling2D, Concatenate, Conv1D, MaxPooling1D
from keras import backend as K
from keras.utils import plot_model
from keras.models import load_model
from sklearn.metrics import classification_report
import random
import os
import librosa
import numpy as np

import time
import datetime
import pytz
import re
import sys, tqdm

import librosa
import librosa.display
import numpy as np
import scipy
import tqdm
import IPython.display as ipd

In [2]:
def smooth(x,window_len=11,window='hanning'):
    """smooth the data using a window with requested size.
    
    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.
    
    input:
        x: the input signal 
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    output:
        the smoothed signal
        
    example:

    t=linspace(-2,2,0.1)
    x=sin(t)+randn(len(t))*0.1
    y=smooth(x)
    
    see also: 
    
    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter
 
    TODO: the window parameter could be the window itself if an array instead of a string
    NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.
    """


    s=np.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    #print(len(s))
    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')

    y=np.convolve(w/w.sum(),s,mode='valid')
    return y

In [3]:
# musica, sr = librosa.load('b9.wav', sr=44100)
# musica=smooth(musica, window_len=20)

In [4]:
# librosa.output.write_wav('gen.wav', musica, sr)

In [5]:
# for i in load_audio('../test', 100):
#     print(i)
#     break

In [28]:
def load_audio(fdir, window_sz=20, batchsz=500):
    for f in os.listdir(fdir):
        musica, sr = librosa.load(fdir+'/'+f, sr=44100)
        smusica=smooth(musica, window_len=20)
        #[win-1-win] is our training window
        xb=[]
        yb=[]
        for i in range(0, musica.shape[0]-1-2*window_sz):
            xb.append(musica[i:i+1+2*window_sz].reshape(-1,1))
            yb.append(musica[i+window_sz+1])
#             print(len(xb))
            if len(xb) == batchsz:
                yield np.array(xb), np.array(yb)
                xb=[]
                yb=[]
        if len(xb)>0:
            yield np.array(xb, dtype=np.float16), np.array(yb, dtype=np.float16)
    #     return np.array(xb), np.array(yb)
#             yield musica[i:i+1+2*window_sz].reshape(1,-1), musica[i+window_sz+1].reshape(1,-1)

In [39]:
window_sz=20
# model = Sequential([
#     Dense(1024, activation='relu', input_shape=(2*window_sz+1,)),
#     Dense(1024, activation='relu'),
#     Dense(1024, activation='relu'),
#     Dense(1, activation='sigmoid'),
# ])
# model.compile(loss=keras.losses.mean_squared_error,
#     optimizer=keras.optimizers.Adam(lr=0.00005, decay=1e-6, beta_1=0.9, beta_2=0.999, amsgrad=False),
#     metrics=['accuracy'])


# regressor = Sequential()

# regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (2*window_sz+1,1)))
# regressor.add(Dropout(0.2))

# regressor.add(LSTM(units = 50, return_sequences = True))
# regressor.add(Dropout(0.2))

# regressor.add(LSTM(units = 50, return_sequences = True))
# regressor.add(Dropout(0.2))

# # regressor.add(LSTM(units = 50))
# # regressor.add(Dropout(0.2))

# regressor.add(Dense(1, activation='sigmoid'))

# regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
# model=regressor

# model = Sequential()
# model.add(Embedding(1, 32, input_length=2*window_sz+1))
# model.add(Bidirectional(LSTM(256)))
# model.add(Dropout(0.5))
# model.add(Dense(1, activation='sigmoid'))
# model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])
# model.build()

model = Sequential()
model.add(Conv1D(32, (3), input_shape=(2*window_sz+1,1), activation='relu'))
model.add(Conv1D(64, (3), activation='relu'))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# model.add(Dense(num_classes, activation='sigmoid'))

model.compile(loss=keras.losses.mean_squared_error,
    optimizer=keras.optimizers.Adam(lr=0.00005, decay=1e-6, beta_1=0.9, beta_2=0.999, amsgrad=False),
    metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_16 (Conv1D)           (None, 39, 32)            128       
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 37, 64)            6208      
_________________________________________________________________
flatten_10 (Flatten)         (None, 2368)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 2369      
Total params: 8,705
Trainable params: 8,705
Non-trainable params: 0
_________________________________________________________________


In [36]:
# for x,y in music_gen('../test'):
#     print(x.shape)

In [41]:
batchsz=10000
train_step=0
test_step=0
dev_step=0
def test():
    predict=[]
    label=[]
    for i in load_audio('../test', batchsz=batchsz):
            loss, acc = model.evaluate(i[0], i[1], verbose=0)
            metrics = {
              'loss_test':loss,
              'accuracy_test':acc
            }
            print(metrics)
#         out=model.predict(i[0])
#         predict+=[i for i in np.argmax(out, axis=1)]
#         label+=[i for i in np.argmax(i[1], axis=1)]
#         sys.stdout.write("T%d| "%test_step)
#         sys.stdout.flush()
    print("====EPOCH====",e,"=====TEST=====")
    print(classification_report(label, predict))

sttime=time.time()
for e in range(0, 10):
#     with experiment.train():
        for loaded in load_audio('../wavdata', batchsz=batchsz):
            train_step+=1
#             print(loaded[0].shape)
#             x=np.abs(loaded[0])
#             x=np.array(x, dtype=np.float16)
#             print(x.reshape(-1, input_shape[0], input_shape[1], input_shape[2]).shape)
#             print(x.dtype)
#             print(loaded[0].shape)
            loss, acc = model.train_on_batch(loaded[0],loaded[1])
            
            metrics = {
                'loss':loss,
                'accuracy':acc
            }
            sys.stdout.write("%d:%.4f-%.4f|"%(train_step, acc, loss))
            sys.stdout.flush()
#             experiment.set_step(train_step)
#             experiment.log_metrics(metrics)

#             if train_step%50==0:
        test()

1:1.0000-0.0013|2:0.8516-0.0013|3:0.1529-0.0013|4:0.0718-0.0013|5:0.0318-0.0013|6:0.0394-0.0013|7:0.0285-0.0012|8:0.0163-0.0012|9:0.0136-0.0012|10:0.0422-0.0012|11:0.0160-0.0012|12:0.0189-0.0012|13:0.0262-0.0012|14:0.0133-0.0012|15:0.0106-0.0012|16:0.0182-0.0012|17:0.0201-0.0012|18:0.0153-0.0012|19:0.0260-0.0012|20:0.0168-0.0012|21:0.0186-0.0011|22:0.0094-0.0012|23:0.0187-0.0011|24:0.0149-0.0011|25:0.0036-0.0014|26:0.0004-0.0028|27:0.0000-0.0022|28:0.0002-0.0020|29:0.0001-0.0018|30:0.0003-0.0015|31:0.0003-0.0014|32:0.0004-0.0013|33:0.0003-0.0012|34:0.0008-0.0013|35:0.0005-0.0012|36:0.0008-0.0013|37:0.0010-0.0013|38:0.0000-0.0012|39:0.0005-0.0012|40:0.0005-0.0012|41:0.0008-0.0012|42:0.0004-0.0011|43:0.0013-0.0011|44:0.0029-0.0010|45:0.0036-0.0010|46:0.0109-0.0010|47:0.0153-0.0010|48:0.0165-0.0010|49:0.0321-0.0010|50:0.0210-0.0010|51:0.0187-0.0010|52:0.0194-0.0010|53:0.0141-0.0010|54:0.0056-0.0023|55:0.0001-0.0032|56:0.0001-0.0026|57:0.0004-0.0017|58:0.0002-0.0015|59:0.0005-0.0014|60:0.0

KeyboardInterrupt: 